In [1]:
import sqlite3

# Определяем класс Professor
class Professor:
    def __init__(self, name, subject, attributes):
        self.name = name
        self.subject = subject
        self.attributes = attributes

# Подключаемся к базе данных (или создаем ее, если она не существует)
conn = sqlite3.connect('professors.db')
cursor = conn.cursor()

# Создаем таблицы для преподавателей и вопросов
cursor.execute('''
CREATE TABLE IF NOT EXISTS questions (
    id INTEGER PRIMARY KEY,
    question TEXT NOT NULL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS professors (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    subject TEXT NOT NULL,
    attributes BLOB NOT NULL
)
''')

# Заполняем таблицу вопросов, если она пуста
cursor.execute('SELECT COUNT(*) FROM questions')
if cursor.fetchone()[0] == 0:
    questions_data = [
        "Мужчина?",
        "Преподает математику?",
        "Преподает физику?",
        "Преподает биологию?",
        "Преподает информатику?",
        "Преподает химию?",
        "Имеет степень магистра?",
        "Имеет степень доктора?",
        "Преподает в университете?"
    ]
    cursor.executemany('INSERT INTO questions (question) VALUES (?)', [(q,) for q in questions_data])

# Заполняем таблицу преподавателей, если она пуста
cursor.execute('SELECT COUNT(*) FROM professors')
if cursor.fetchone()[0] == 0:
    professors_data = [
        ("Преподаватель по математике 1", "математика", [1, 1, 0, 0, 0, 0, 1, 0, 0]),
        ("Преподаватель по математике 2", "математика", [1, 0, 0, 0, 0, 0, 0, 1, 0]),
        ("Преподаватель по физике 1", "физика", [1, 0, 1, 0, 0, 0, 0, 1, 0]),
        ("Преподаватель по физике 2", "физика", [1, 0, 1, 0, 0, 0, 0, 0, 1]),
        ("Преподаватель по биологии 1", "биология", [0, 0, 0, 0, 1, 0, 0, 1, 0]),
        ("Преподаватель по биологии 2", "биология", [0, 0, 0, 0, 1, 0, 0, 0, 1]),
        ("Преподаватель по химии 1", "химия", [1, 0, 0, 0, 0, 1, 0, 1, 0]),
        ("Преподаватель по химии 2", "химия", [0, 1, 0, 0, 0, 1, 0, 0, 1]),
        ("Преподаватель по информатике 1", "информатика", [1, 0, 0, 0, 0, 1, 0, 1, 0]),
        ("Преподаватель по информатике 2", "информатика", [1, 0, 0, 0, 0, 1, 0, 0, 1]),
    ]
    
    for name, subject, attributes in professors_data:
        cursor.execute('INSERT INTO professors (name, subject, attributes) VALUES (?, ?, ?)', 
                       (name, subject, str(attributes)))

def get_questions():
    cursor.execute('SELECT question FROM questions')
    return [row[0] for row in cursor.fetchall()]

def get_professors():
    cursor.execute('SELECT name, subject, attributes FROM professors')
    return [Professor(name=row[0], subject=row[1], attributes=eval(row[2])) for row in cursor.fetchall()]

def ask_question(professors, remaining_questions):
    if not remaining_questions:
        return professors

    question_index = remaining_questions[0]
    question = get_questions()[question_index]
    answer = input(f"{question} (да/нет): ").strip().lower()

    if answer == 'да':
        filtered_professors = [prof for prof in professors if prof.attributes[question_index] == 1]
        # Если ответ "да" на вопрос о предмете, удаляем соответствующие вопросы о других предметах
        if question == "Преподает математику?":
            remaining_questions = [0, 6, 7, 8]  # Сохраняем только вопросы о магистре, докторе и университете
        elif question == "Преподает физику?":
            remaining_questions = [1, 6, 7, 8]  # Аналогично для физики
        elif question == "Преподает биологию?":
            remaining_questions = [2, 6, 7, 8]  # Аналогично для биологии
        elif question == "Преподает информатику?":
            remaining_questions = [3, 6, 7, 8]  # Аналогично для информатики
        elif question == "Преподает химию?":
            remaining_questions = [4, 6, 7, 8]  # Аналогично для химии
    elif answer == 'нет':
        filtered_professors = [prof for prof in professors if prof.attributes[question_index] == 0]
    else:
        print("Пожалуйста, отвечайте только 'да' или 'нет'.")
        return ask_question(professors, remaining_questions)  # Повторяем вопрос

    if len(filtered_professors) == 0:
        print("Преподаватель не найден. Предлагаем внести нового преподавателя в базу данных.")
        add_professor()
    elif len(filtered_professors) == 1:
        print(f"Вы выбрали: {filtered_professors[0].name} (предмет: {filtered_professors[0].subject})")
    else:
        # Убираем заданный вопрос из оставшихся вопросов
        new_remaining_questions = remaining_questions[1:]
        ask_question(filtered_professors, new_remaining_questions)  # Задаем следующий вопрос

def add_professor():
    name = input("Введите имя нового преподавателя: ")
    subject = input("Введите предмет, который будет преподавать: ").strip()
    attributes = []
    
    for question in get_questions():
        answer = input(f"{question} (да/нет): ").strip().lower()
        if answer == 'да':
            attributes.append(1)
        elif answer == 'нет':
            attributes.append(0)
        else:
            print("Пожалуйста, отвечайте только 'да' или 'нет'.")
            return add_professor()  # Повторяем добавление

    cursor.execute('INSERT INTO professors (name, subject, attributes) VALUES (?, ?, ?)', 
                   (name, subject, str(attributes)))
    print(f"{name} успешно добавлен в базу данных!")

# Запускаем Акинатора
print("Добро пожаловать в Акинатора! Давайте попробуем угадать преподавателя.")
questions = get_questions()
professors = get_professors()
ask_question(professors, list(range(len(questions))))  # Начинаем с всех вопросов

# Закрываем соединение с базой данных
conn.commit()  # Сохраняем изменения
conn.close()

Добро пожаловать в Акинатора! Давайте попробуем угадать преподавателя.
Преподаватель не найден. Предлагаем внести нового преподавателя в базу данных.
